In [1]:
%%configure -f
{ "conf":{
          "spark.jars": "s3://stoic-quant-raw-data/postgresql-42.2.5.jar"
         }
}

In [2]:
df = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://ls-dd31fddee5fb278b7636d965e5004e29de9a083b.cj4kyg10bbsp.ap-south-1.rds.amazonaws.com:5432/stoicim_research") \
    .option("user", "postgres") \
    .option("password", "stoicimresearch123") \
    .option("driver", "org.postgresql.Driver") \
    .option("query", "SELECT a.*, b.name from weekly_securities_prices a inner join securities b on b.accord_code = a.accord_code") \
    .load()

#df.count()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
14,application_1669534397620_0015,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
from pyspark.sql.functions import countDistinct
df2 = df.select(countDistinct("accord_code"))
df2.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------------+
|count(DISTINCT accord_code)|
+---------------------------+
|                       7002|
+---------------------------+

In [4]:
from pyspark.sql.functions import col, avg, first, stddev, row_number, monotonically_increasing_id

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
df_price = df.select(col("accord_code"), col("name"), col("tdate"), col("adj_close"), col("market_cap"), col("volume"), col("no_of_trades"), col("delivery_volume"), col("delivery_percent"))
df_price.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[accord_code: bigint, name: string, tdate: timestamp, adj_close: double, market_cap: double, volume: double, no_of_trades: double, delivery_volume: double, delivery_percent: double]

In [6]:
df_price.show(n=10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+------------------+-------------------+---------+------------+-----------------+------------+---------------+----------------+
|accord_code|              name|              tdate|adj_close|  market_cap|           volume|no_of_trades|delivery_volume|delivery_percent|
+-----------+------------------+-------------------+---------+------------+-----------------+------------+---------------+----------------+
|     131131|Mascon Global Ltd.|2003-05-05 00:00:00|     9.75|34.402324125|          117.328|       598.0|            0.0|             0.0|
|     131131|Mascon Global Ltd.|2003-05-12 00:00:00|      9.6|  33.8730576|          184.752|       821.0|            0.0|             0.0|
|     131131|Mascon Global Ltd.|2003-05-19 00:00:00|     8.75|30.873880625|          229.895|       877.0|            0.0|             0.0|
|     131131|Mascon Global Ltd.|2003-05-26 00:00:00|      8.5| 29.99176975|          182.806|       868.0|            0.0|             0.0|
|     131131|Mascon 

In [7]:
#variables
time_window_for_blended_price = 0  #rolling average price of no of weeks
time_window_for_evaluation = 52 #no of weeks for which to evaluate price growth
time_window_for_historic_trade_volume = 52
time_window_for_current_trade_volume = 4
trade_volume_growth = 3
rebalance_frequency = 52 #every n week

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
from pyspark.sql import functions as f
from pyspark.sql.window import Window

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
last_n_rows = Window.partitionBy('accord_code').orderBy('tdate').rowsBetween(-time_window_for_blended_price, 0)
df_price = df_price.withColumn("blended_price",avg(df_price.adj_close).over(last_n_rows))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
last_n_rows = Window.partitionBy('accord_code').orderBy('tdate').rowsBetween(-time_window_for_evaluation, 0)
df_price = df_price.withColumn("last_price",first(df_price.blended_price).over(last_n_rows))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
last_n_rows = Window.partitionBy('accord_code').orderBy('tdate').rowsBetween(-time_window_for_evaluation, 0)
df_price = df_price.withColumn("std_dev",stddev(df_price.blended_price).over(last_n_rows))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
last_n_rows = Window.partitionBy('accord_code').orderBy('tdate').rowsBetween(-time_window_for_historic_trade_volume, 0)
df_price = df_price.withColumn("historic_volume",avg(df_price.volume).over(last_n_rows))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
last_n_rows = Window.partitionBy('accord_code').orderBy('tdate').rowsBetween(-time_window_for_current_trade_volume, 0)
df_price = df_price.withColumn("current_volume",avg(df_price.volume).over(last_n_rows))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
#df_price.select(col("name"), col("tdate") ,col("blended_price"), col("last_price")).show(n=10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
df_price = df_price.withColumn('growth', (col("blended_price") - col("last_price")) / col("last_price") * 100)
#df_price.select(col("name"), col("tdate") ,col("blended_price"), col("last_price"), col("growth"), col("std_dev")).show(n=10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
df_price = df_price.withColumn('volume_growth', (col("current_volume") - col("historic_volume")) / col("historic_volume") * 100)
#df_price.select(col("name"), col("tdate") ,col("blended_price"), col("last_price"), col("growth"), col("std_dev")).show(n=10)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
df_price = df_price.filter(df_price.volume_growth > 3)
last_n_rows = Window.partitionBy('tdate').orderBy(col('growth').desc(), col('std_dev'))
df_price = df_price.withColumn("row",row_number().over(last_n_rows)).filter(col("row") <= 20).drop("row")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:

df_rebalance_dates = df_price.select(col("tdate")).dropDuplicates().orderBy(col('tdate'))
df_rebalance_dates = df_rebalance_dates.withColumn("index", row_number().over(Window.orderBy(monotonically_increasing_id()))).filter(col("index") % rebalance_frequency == 0)
df_rebalance_dates = df_rebalance_dates.drop(col('index'))
#df_rebalance_dates.show(n=50)



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
df_portfolio_log = df_price.join(df_rebalance_dates,df_price.tdate ==  df_rebalance_dates.tdate,"inner")
#df_portfolio_log.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
df_portfolio_log.show(n=800)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------------------+-------------------+-----------------+---------------+--------------------+------------+-------------------+----------------+-----------------+------------------+--------------------+--------------------+-------------------+------------------+------------------+-------------------+
|accord_code|                name|              tdate|        adj_close|     market_cap|              volume|no_of_trades|    delivery_volume|delivery_percent|    blended_price|        last_price|             std_dev|     historic_volume|     current_volume|            growth|     volume_growth|              tdate|
+-----------+--------------------+-------------------+-----------------+---------------+--------------------+------------+-------------------+----------------+-----------------+------------------+--------------------+--------------------+-------------------+------------------+------------------+-------------------+
|     131411|Tuni Textile Mill...|2010-12-20 00:0

In [25]:
df_price_snap = df_price.filter(col("tdate").between(start_date_1,start_date_2)) 
#df_price_snap.select(col("name"), col("tdate") ,col("blended_price"), col("last_price"), col("growth"), col("std_dev"), col("current_volume"),col("historic_volume"),col("volume_growth")).show(10)
df_folio = df_price_snap.orderBy("growth", ascending=False).limit(50)
df_folio = df_folio.orderBy("std_dev")
df_folio = df_folio.filter(df_folio.volume_growth > 3).limit(20)
df_folio.select(col("name"), col("tdate") ,col("blended_price"), col("last_price"), col("growth"), col("std_dev"), col("current_volume"),col("historic_volume"),col("volume_growth")).show(50)                                
                                

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'start_date_1' is not defined
Traceback (most recent call last):
NameError: name 'start_date_1' is not defined



In [26]:
from datetime import datetime, date, timedelta
df_folio_history = None
while True:
    print("Rebalancing on : ", start_date_1)
    df_price_snap = df_price.filter(col("tdate").between(start_date_1,start_date_2)) 
    if df_price_snap.count == 0:
        break
    df_folio = df_price_snap.orderBy("growth", ascending=False).limit(50)
    df_folio = df_folio.orderBy("std_dev")
    df_folio = df_folio.filter(df_folio.volume_growth > 3).limit(20)
    df_folio = df_folio.select(col("name"), col("tdate") ,col("blended_price"), col("last_price"), col("growth"), col("std_dev"), col("current_volume"),col("historic_volume"),col("volume_growth"))                               
    if df_folio_history == None:
        print("portfolio created!")
        df_folio_history = df_folio
    else:
        df_folio_history = df_folio_history.union(df_folio)
    
    start_date_1 = (datetime.strptime(start_date_1, '%Y-%m-%d %H:%M:%S') + timedelta(days=rebalance_frequency)).strftime('%Y-%m-%d %H:%M:%S')
    start_date_2 = (datetime.strptime(start_date_2, '%Y-%m-%d %H:%M:%S') + timedelta(days=rebalance_frequency)).strftime('%Y-%m-%d %H:%M:%S')    
    
    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'start_date_1' is not defined
Traceback (most recent call last):
NameError: name 'start_date_1' is not defined

